<a href="https://colab.research.google.com/github/harshakoneru98/city_watch/blob/main/data_processing/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install uszipcode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 KB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for atomicwrites: filename=atomicwrites-1.4.1-py2.py3-none-any.whl size=6955 sha256=cbc6ed865b28130c99a09a9b13ddc24cd322ec46be25780ef17f5680bf0d37df
  Stored in directory: /root/.cache/pip/wheels/2a/b5/06/d2f34584f352e4af7d1d7ac1baf38b5d24142c8044cd024fd5
Successfully built atomicwrites


In [ ]:
# importing libraries
import pandas as pd
import json
import tqdm
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
ROOT_PATH = "/content/drive/MyDrive/DSCI 560/Datasets/"

In [ ]:
# common util to get necessary columns
def get_filtered_columns(data, key_names):
  filtered_data = []
  for record in data:
    row = {}
    for key in key_names:
      r = record
      for key_comp in key.split('.')[:-1]:
        r = r.get(key_comp, {})
      row[key] = r.get(key.split('.')[-1], None)
    filtered_data.append(row)
  return filtered_data

In [ ]:
# loading business data
business_json = json.load(open(f"{ROOT_PATH}/business.json"))
print(len(business_json))
key_names = ['business_name', 'street_address', 'city', 'zip_code', 
             'location_start_date', 'location_end_date', 'location_description']
business_df = pd.DataFrame(get_filtered_columns(business_json, key_names))
business_df.loc[business_df.location_end_date.isna(), 'location_end_date'] = pd.to_datetime('12-31-2023') # setting end date of currently running businesses to year end.
business_df.location_end_date = pd.to_datetime(business_df.location_end_date, errors='coerce')
business_df = business_df.dropna() # dropping null values
business_df.location_start_date = pd.to_datetime(business_df.location_start_date, errors='coerce') # changing dates from string to datetime format. 
business_df.city = business_df.city.str.lower() # making everything lowercase
del business_json

In [ ]:
# loading housing data
housing_json = json.load(open(f"{ROOT_PATH}/housing.json"))
print(len(housing_json))
key_names = ['issue_date', 'street_name', 'work_description', 'zip_code', 'valuation',
             'of_residential_dwelling_units', 
             'floor_area_l_a_zoning_code_definition', 
             'of_stories', 'census_tract', 'floor_area_l_a_building_code_definition',
             ]
housing_df = pd.DataFrame(get_filtered_columns(housing_json, key_names)).dropna()
housing_df.issue_date = pd.to_datetime(housing_df.issue_date, errors='coerce') # changing dates from string to datetime format.
del housing_json

25013


In [ ]:
# aligning dates. We do not want businesses that closed before the earliest record of houses.
business_df_date = business_df[business_df.location_end_date >= housing_df.issue_date.min()] 

In [ ]:
# saving business dataset
business_df.to_csv(f"{ROOT_PATH}/Cleaned_data/businesses.csv")
business_df_date.to_csv(f"{ROOT_PATH}/Cleaned_data/businesses_date_aligned.csv")

In [ ]:
# We do not have city in housing data. So we use the zip code and city in businesses data to fill that column
zip_city = {k:v for k, v in business_df[['zip_code', 'city']].values.tolist()}
zc2 = {}
for key in zip_city:
  zc2[key.split("-")[0]] = zip_city[key]
housing_df['city'] = housing_df.apply(lambda x: zc2.get(x.zip_code), axis=1)
print(housing_df.isna().sum())
# there are no null values in city

In [ ]:
# saving housing dataset
housing_df.to_csv(f"{ROOT_PATH}/Cleaned_data/housing.csv")

In [ ]:
# loading crime data ( part 1)
crime1_json = json.load(open(f"{ROOT_PATH}/crime10_19.json"))
print(len(crime1_json))
key_names = list(crime1_json[0].keys())
key_names = ['dr_no', 'date_rptd', 'date_occ', 'area', 'area_name', 'rpt_dist_no',
            'crm_cd', 'crm_cd_desc', 'vict_age', 'vict_sex', 'vict_descent', 'vict_descent', 'location', 'lat', 'lon']
crime_df = pd.DataFrame(crime1_json, columns=key_names)
crime_df.date_rptd = pd.to_datetime(crime_df.date_rptd, errors='coerce') # changing dates from string to datetime format. 
crime_df.date_occ = pd.to_datetime(crime_df.date_occ, errors='coerce') # changing dates from string to datetime format. 
crime_df = crime_df.dropna() # dropping null values
del crime1_json

2119797


In [ ]:
# loading crime data ( part 2)
crime2_json = json.load(open(f"{ROOT_PATH}/crime20_23.json"))
print(len(crime2_json))
key_names = ['dr_no', 'date_rptd', 'date_occ', 'area', 'area_name', 'rpt_dist_no',
            'crm_cd', 'crm_cd_desc', 'vict_age', 'vict_sex', 'vict_descent', 'vict_descent', 'location', 'lat', 'lon']
crime2_df = pd.DataFrame(crime2_json, columns=key_names)
crime2_df.date_rptd = pd.to_datetime(crime_df.date_rptd, errors='coerce') # changing dates from string to datetime format. 
crime2_df.date_occ = pd.to_datetime(crime_df.date_occ, errors='coerce') # changing dates from string to datetime format. 
crime2_df = crime2_df.dropna() # dropping null values
crime_df = pd.concat([crime_df, crime2_df]) # appending both crime data
del crime2_df
del crime2_json

638245


In [ ]:
_# we do not have zipcode data in crime dataset. So, we are using uszipcode to fill in the information
from uszipcode import SearchEngine
sr = SearchEngine()

def get_zipcode(lat, long):
  record = sr.by_coordinates(float(lat), float(long), returns=1, radius=5)
  if len(record) > 0:
    return record[0].zipcode
  else:
    return None
crime_df['zip_code'] = crime_df.apply(lambda x: get_zipcode(float(x.lat), float(x.lon)), axis=1)
crime_df = crime_df.dropna()

In [ ]:
# aligning dates. We do not want crimes that closed before the earliest record of houses.
crime_df_date = crime_df[crime_df.date_occ >= housing_df.issue_date.min()]

In [ ]:
len(crime_df)

1922982

In [ ]:
# saving crime dataset
crime_df.to_csv(f"{ROOT_PATH}/Cleaned_data/crime.csv")
crime_df_date.to_csv(f"{ROOT_PATH}/Cleaned_data/crime_date_aligned.csv")